In [31]:
from dotenv import load_dotenv
from agents import Agent, Runner,trace, OpenAIChatCompletionsModel, OutputGuardrail, GuardrailFunctionOutput,OutputGuardrailTripwireTriggered
from openai import AsyncOpenAI
from pydantic import BaseModel
import os
#print(os.getenv("OPENROUTER_API_KEY"))

In [32]:
#Create the client
client = AsyncOpenAI(base_url="https://openrouter.ai/api/v1",
                      api_key=os.getenv("OPENROUTER_API_KEY"))

In [33]:
#Create Open AI Chat Completaion Model
openroutermodel= OpenAIChatCompletionsModel(openai_client=client, model="tngtech/deepseek-r1t2-chimera:free")

In [34]:
class MessageOutput(BaseModel):
    response:str

class Output(BaseModel):
    is_valid: bool
    response: str

outputvalidate = Agent(name="outputvalidate",
                       instructions="check if joke is on Math. if yes than set is_valid to true.",
                       model="gpt-4.1-mini",
                        output_type=Output)

@OutputGuardrail
async def validateResponse(cxt, agent, input):
    respose = await Runner.run(starting_agent=outputvalidate, 
                               context=cxt.context,
                               input=input)
    return GuardrailFunctionOutput(output_info=respose.final_output, 
                                   tripwire_triggered=respose.final_output.is_valid)
    
    




In [35]:
#Create AIAgent
aiagent = Agent(name="Joker", instructions="Tell me a joke",
              model="gpt-4.1-mini", output_guardrails=[validateResponse])

In [36]:
with trace("Joke"):
    try:
        response = await Runner.run(aiagent,"Tell me joke on AI")
        print(response.final_output)
    except OutputGuardrailTripwireTriggered:
        print("Guardrail has been tiggered")

Sure! Here's an AI joke for you:

Why did the AI go to therapy?

Because it had too many neural issues!
